In [1]:
!pip install jellyfish
!pip install faker

In [1]:
import pyspark
import jellyfish
import pandas as pd
import numpy as np
from typing import List
import os
import math
from itertools import combinations, product
import time
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict
from packages.generateDataSets import SyntheticMatcherDataset
from packages.calculateStatistics import DatasetEvaluator

In [2]:
class MyClass:
    def __init__(self, df1: pd.DataFrame, 
                 df2: pd.DataFrame, 
                 matchColumn: str, 
                 on: List = [],
                 method: str = 'column', 
                 threshold: float  = 0.6):
        self.df1 = df1
        self.df2 = df2
        self.on = on
        self.threshold = threshold

        if method not in ["concat", "column"]:
            raise ValueError(f"Method '{method}' is not correct.")
        self.method = method

    
        if matchColumn not in self.df1.columns or matchColumn not in self.df2.columns:
            raise ValueError(f"Column '{matchColumn}' is not found in both DataFrames.")
        self.matchColumn = matchColumn
        
        self.groundTruth = None
        self.totalMatches = None        
    
    def setGroundTruth(self):
        """Sets the ground truth based on matching 'id' columns."""
        self.groundTruth = np.intersect1d(self.df1[self.matchColumn], self.df2[self.matchColumn])

    def soundexDfs(self):
        """Apply soundex transformation to non-id columns."""
        for df in [self.df1, self.df2]:
            for col_name in df.columns:
                if col_name != self.matchColumn:
                    df[col_name] = df[col_name].apply(lambda x: jellyfish.soundex(str(x)))

            if self.method == 'concat':
                non_match_columns = [col for col in df.columns if col != self.matchColumn]
                df['concatenated'] = df[non_match_columns].apply(lambda row: ''.join(row.astype(str)), axis=1)
                df.drop(columns=non_match_columns, inplace=True)

    def setTotalMatches(self):
        """Sets the total matches based on merged DataFrames."""
        
        # if self.method == 'concat':
        #     self.totalMatches = self.df1.merge(self.df2, how="inner", on=['concatenated']).to_numpy()
        # else:   
        #     self.totalMatches = self.df1.merge(self.df2, how="outer", on=self.on + [self.matchColumn]).to_numpy()

        self.totalMatches =  self.df1.merge(pd.concat([self.df1, self.df2]), how='outer', on=self.on)[["0_y"] + self.on]
        self.totalMatches.rename(columns={'0_y': self.matchColumn}, inplace=True)
        
    def printStatistics(self):
        """Print statistics (True Positives, False Positives, Precision)."""
        myStatistics = self.Statistics(groundTruth=self.groundTruth, 
                                       totalMatches=self.totalMatches, 
                                       threshold=self.threshold, 
                                       on=self.on, 
                                       matchColumn=self.matchColumn)
        myStatistics.calculate()

    # Inner class Statistics
    class Statistics:
        def __init__(self,
                     groundTruth: pd.DataFrame, 
                     totalMatches: pd.DataFrame, 
                     threshold : float = 0.8,
                     matchColumn: str | int = 1,
                     on: List =[]):
            self.groundTruth = pd.DataFrame(groundTruth)
            self.totalMatches = pd.DataFrame(totalMatches)
            self.threshold = threshold
            self.matchColumn = matchColumn
            self.on = on

            self._setThresholdValues()
            
        def calculate(self):
            # self.result = self.totalMatches.groupby(self.matchColumn)\
            #         .filter(lambda x : len(x) >=2)\
            #         .groupby(self.matchColumn)\
            #         .apply(lambda x: x.iloc[:, 1:].apply(lambda x: x.nunique() == 1)).sum(axis=1)

            duplicates = self.totalMatches[self.totalMatches[[1,2,3,4,5]].duplicated(keep=False)].sort_values(by=[1,2,3,4,5])
        
            # Function to check if two rows match at least 3/5 columns
            def is_duplicate(row1, row2):
                return sum(row1 == row2) >=  self.matchingRows  # At least 3 matches out of 5

            # print(duplicates)
            
            duplicate_pairs = []
            # Find duplicates
            for i in range(len(duplicates)):
                for j in range(i + 1, len(duplicates)):  # Compare only unique pairs
                    if is_duplicate(duplicates.iloc[i, 1:], duplicates.iloc[j, 1:]):
                        duplicate_pairs.append((i, j, duplicates.iloc[i, 0] == duplicates.iloc[j, 0]))  # Store (index1, index2, same_id)

            # Count same ID and different ID duplicates
            tp = sum(1 for _, _, same_id in duplicate_pairs if same_id)
            fp = len(duplicate_pairs) - same_id_count
            fn = self.groundTruth.size - tp
            
            precision = tp / (tp + fp) if tp + fp != 0 else 0 
            recall = tp / (tp + fn)  if tp + fn != 0 else 0
            f1_score = (2 * precision * recall) / (precision + recall)
            
            print("Total Possible Mathces:", self.groundTruth.size)
            print("True Positives (TP):", tp)
            print("False Positives (FP):", fp)
            print("False Negatives (FN):", fn)
            print("Precision:", f"{precision:.4f}")
            print("Recall:", f"{recall:.4f}")
            print("F1-score:", f"{f1_score:.4f}")

        def _matchingAlgorithm(self, group):
            return group.nunique() == 1
            
        def _setThresholdValues(self) -> List:
            size = len(self.totalMatches.columns) - 1
            limit = math.floor(self.threshold * size)
            
            print(f"We accept at least {limit}/{size} as matches!") 
            self.matchingRows = limit
            # return [i for i in range(size, limit , -1)]
            

In [3]:
PATH  =  "data/"

df1 = pd.read_csv(os.path.join(PATH, 'df1.csv'), header=None)[[0,1,2,3,4,5]]
df2 = pd.read_csv(os.path.join(PATH, 'df5.csv'), header=None)[[0,1,2,3,4,5]]

# Run pipeline and see statistics
pipeline = MyClass(df1, df2, matchColumn=0, on=[1,2,3,4,5], method="column", threshold = 0.6) #  --> this means at least 3/5 of the fields must match 
pipeline.setGroundTruth()
pipeline.soundexDfs()

In [8]:
evaluator = DatasetEvaluator(pipeline.df1, pipeline.df2, threshold=4)
evaluator.evaluate()
evaluator.printResults()

KeyboardInterrupt: 

In [ ]:
thresh = 2
Expected: {}
Ground Truth Size: 25000
True Positives: 24977
False Positives: 74055
False Negatives: 23
Precision: 0.2522
Recall: 0.9991
Elapsed Time: 4861.54 seconds

thresh = 3
Expected: {}
Ground Truth Size: 25000
True Positives: 19125
False Positives: 16323
False Negatives: 5875
Precision: 0.5395
Recall: 0.7650
Elapsed Time: 80256.47 seconds

In [1]:
import pyspark
import jellyfish
import pandas as pd
import numpy as np
from typing import List
import os
import math
from itertools import combinations, product
import time
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict
from packages.generateDataSets import SyntheticMatcherDataset
from packages.calculateStatistics import DatasetEvaluator

In [68]:
# Create two datasets with slight variations
# Data have 3 matches and one 
data1 = {
    0: [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    1: ["Kostas", "Maria", "John", "Sophia", "George", "Eleni", "Michael", "Anna", "Chris", "Dimitris"],
    2: ["Razgkelis", "Papadopoulos", "Smith", "Johnson", "Pavlou", "Nikolaou", "Brown", "Miller", "Taylor", "Andreas"],
    3: ["Orestiada", "Thessaloniki", "Grevena", "Athina", "Aleksandroupoli", "Giannena", "Larissa", "Komotini", "Trikala", "Kozani"],
    4: ['Jennifer Lights', 'Brandon Lakes', 'Aguilar Stravenue', 'Richardson Ferry', 'Freeman Way', 
        'Gabrielle Underpass', 'Burns Summit', 'Heather Village', 'Jamie Common', 'Greg Lock'],
    5:  ['Cooper and Sons', 'Pope LLC', 'Fowler-Smith', 'Torres PLC', 'Jones LLC', 'White, Duncan and Robinson', 'Hayden Inc', 
         'Wilson and Sons', 'Peterson, Smith and Robinson','Hudson, Phelps and Day'],
    
}

data2 = {
    0: [101, 202, 203, 204, 205, 206, 207, 208, 209, 110],
    1: ["Kistas", "Maria", "John", "Sophasdia", "Giorge", "Elendsi", "Micheal", "Ana", "Khris", "Dimtris"],
    2: ["Rozgkliiis", "Papadopoulos", "Smith", "Johnson", "Pavlodvu", "Nikolaou", "Batrrroun", "antMiler", "Tttayloor", "Andres"],
    3: ["Orestiada", "Thessaloniki", "Grevena", "Athina", "Aleksandrouasdpoli", "Gianasdna", "Larasdissa", "Komasdini", "Trsadala", "Koxani"],
    4: ["Jnnfer Lights", 'Brandon Lakes', 'Aguilar Stravenue', 'RichardasasdaFerry', 'Freasdeman Way', 'Gabrielle Underpass', 'Burasdas mmit', 'Heatasasdllage', 'JamasdCommon', 'Grg Lck'],
    5:  ['Cpeeer and ons', 'Pope LLC', 'Fowler-Smith', 'Torvasd PLC', 'Jonasda LLC', 'Whitasddvuncan and Robinson', 'Hayasdasv Inc', 
         'Wasdvand Sons', 'Petersosdvaith and Robinson','Htsn, Phelps and Day'],
}




In [114]:

# Convert to DataFrame
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Run pipeline and see statistics
pipeline = MyClass(df1, df2, matchColumn=0, on=[1, 2, 3, 4, 5], threshold=0.4)
pipeline.setGroundTruth()
pipeline.soundexDfs()
pipeline.setTotalMatches()
# pipeline.printStatistics()

0    101
Name: 0, dtype: int64
0    110
Name: 1, dtype: int64


In [244]:
PATH  =  "data/"

df1 = pd.read_csv(os.path.join(PATH, 'df1.csv'), header=None)[[0,1,2,3,4,5]]
df2 = pd.read_csv(os.path.join(PATH, 'df5.csv'), header=None)[[0,1,2,3,4,5]]

# Run pipeline and see statistics
pipeline = MyClass(df1, df2, matchColumn=0, on=[1,2,3,4,5], method="column", threshold = 0.6) #  --> this means at least 3/5 of the fields must match 
pipeline.setGroundTruth()
pipeline.soundexDfs()
# pipeline.setTotalMatches()
# pipeline.printStatistics()

In [53]:
def is_similar(row1, row2, threshold=4):
    return np.sum(row1[1:] == row2[1:]) >= threshold  # Compare columns 1-5


# Function to process chunks
def process_chunk(chunk_row_pairs, df1, df2, ground_truth):
    totalMatches = []
    for id1, id2 in chunk_row_pairs:
        if is_similar(df1[df1[:, 0] == id1][0], df2[df2[:, 0] == id2][0], 3):
            totalMatches.append((id1, id2))

    # Calculate tp, fp, fn for this chunk
    tp = sum(1 for x, y in totalMatches if x == y and x in ground_truth)
    fp = len(totalMatches) - tp
    fn = len(ground_truth) - tp
    
    return tp, fp, fn



start_time = time.time()

size = 200

df1 = pd.concat([pipeline.df1.iloc[:size], pipeline.df1.iloc[25_000:25000 + size * 3]]).to_numpy()
df2 = pd.concat([pipeline.df2.iloc[:size], pipeline.df2.iloc[25_000:25000 + size * 3]]).to_numpy()

ground_truth = np.intersect1d(df1[:, 0], df2[:, 0])

# Split row pairs into chunks
chunk_size = 1000  # Adjust chunk size based on memory and performance

# Process in chunks
total_tp, total_fp, total_fn = 0, 0, 0
for i in range(0, len(df1), chunk_size):
    # Generate row pairs for the current chunk
    chunk_row_pairs = list(product(df1[i:i + chunk_size, 0], df2[:, 0]))
    tp, fp, fn = process_chunk(chunk_row_pairs, df1, df2, ground_truth)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn

# Calculate precision and recall
precision = total_tp / (total_tp + total_fp) if total_tp + total_fp > 0 else 0
recall = total_tp / (total_tp + total_fn) if total_tp + total_fn > 0 else 0

elapsed_time = time.time() - start_time

print(f"True Positives: {total_tp}")
print(f"False Positives: {total_fp}")
print(f"False Negatives: {total_fn}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

True Positives: 133
False Positives: 5
False Negatives: 67
Precision: 0.9638
Recall: 0.6650
Elapsed Time: 52.30 seconds


In [21]:
pd.concat([pipeline.df1.iloc[:250], pipeline.df1.iloc[25_000:25000 + 750]]).to_numpy().size

6000

In [40]:
sum([7.85, 8.35, 9.89, 8.78, 8.87])

1745433922.1039279

In [245]:
def is_similar(row1, row2, threshold=3):
    return np.sum(row1[1:] == row2[1:]) >= threshold

def process_chunk(chunk_row_pairs, df1_dict, df2_dict, ground_truth, threshold=3):
    total_matches = []
    for id1, id2 in chunk_row_pairs:
        row1 = df1_dict.get(id1)
        row2 = df2_dict.get(id2)
        if row1 is not None and row2 is not None:
            if is_similar(row1, row2, threshold):
                total_matches.append((id1, id2))

    tp = sum(1 for id1, id2 in total_matches if id1 == id2 and id1 in ground_truth)
    fp = len(total_matches) - tp
    
    return tp, fp
    
# Preprocess
size = 500
df1 = pd.concat([pipeline.df1.iloc[:size], pipeline.df1.iloc[25000:25000 + size * 3]]).to_numpy()
df2 = pd.concat([pipeline.df2.iloc[:size], pipeline.df2.iloc[25000:25000 + size * 3]]).to_numpy()

df1_dict = {row[0]: row for row in df1}
df2_dict = {row[0]: row for row in df2}
ground_truth = set(np.intersect1d(df1[:, 0], df2[:, 0]))

chunk_size = 500
total_tp = total_fp = total_fn = 0

start_time = time.time()

for i in range(0, len(df1), chunk_size):
    chunk_ids = df1[i:i + chunk_size, 0]
    chunk_row_pairs = list(product(chunk_ids, df2[:, 0]))
     tp, fp = process_chunk(chunk_row_pairs, df1_dict, df2_dict, ground_truth)
    total_tp += tp
    total_fp += fp

total_fn = len(ground_truth) - total_tp

precision = total_tp / (total_tp + total_fp) if total_tp + total_fp > 0 else 0
recall = total_tp / (total_tp + total_fn) if total_tp + total_fn > 0 else 0

elapsed_time = time.time() - start_time


print(f"True Positives: {total_tp}")
print(f"False Positives: {total_fp}")
print(f"False Negatives: {total_fn}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

True Positives: 330
False Positives: 50
False Negatives: 170
Precision: 0.8684
Recall: 0.6600
Elapsed Time: 26.34 seconds


In [ ]:
148.94

In [62]:
len(ground_truth)

2000

In [ ]:
n = 5000

True Positives: 3437
False Positives: 1697
False Negatives: 1563
Precision: 0.6695
Recall: 0.6874
Elapsed Time: 2599.92 seconds

n = 15000

True Positives: 9903
False Positives: 8710
False Negatives: 5097
Precision: 0.5320
Recall: 0.6602
Elapsed Time: 26709.04 seconds

In [333]:
# Index df2 by a hash on a few features to reduce comparisons
# For example, use a subset of columns to pre-group similar rows
# Here, we use the first column (ID) as a rough bucket
from collections import defaultdict
import time
import numpy as np
import pandas as pd

def is_similar(row1, row2, threshold=2):
    return np.sum(row1[1:] == row2[1:]) >= threshold

# Preprocess
size = 500
df1 = pd.concat([pipeline.df1.iloc[:size], pipeline.df1.iloc[25000:25000 + size * 3]]).to_numpy()
df2 = pd.concat([pipeline.df2.iloc[:size], pipeline.df2.iloc[25000:25000 + size * 3]]).to_numpy()

ground_truth_ids = set(np.intersect1d(df1[:,0], df2[:,0]))

df2_buckets = defaultdict(list)
for row in df2:
    df2_buckets[row[0]].append(row)  # You can change the key to a composite or substring

start_time = time.time()

matched = set()

for row1 in df1:
    candidates = df2_buckets.get(row1[0], [])
    for row2 in candidates:
        if is_similar(row1, row2, threshold=3):
            matched.add((row1[0], row2[0]))  # Track ID pairs
            break

total_tp = sum(1 for id1, id2 in matched if id1 == id2 and id1 in ground_truth_ids)
total_fp = sum(1 for id1, id2 in matched if id1 != id2 or id1 not in ground_truth_ids)
total_fn = len(ground_truth_ids) - total_tp 

precision = total_tp / (total_tp + total_fp) if total_tp + total_fp > 0 else 0
recall = total_tp / (total_tp + total_fn) if total_tp + total_fn > 0 else 0

elapsed_time = time.time() - start_time

print(f"True Positives: {total_tp}")
print(f"False Positives: {total_fp}")
print(f"False Negatives: {total_fn}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

InvalidIndexError: (slice(None, None, None), 0)

In [ ]:
matched

In [5]:
PATH  =  "data/"

df1 = pd.read_csv(os.path.join(PATH, 'df1.csv'), header=None)[[0,1,2,3,4,5]]
df2 = pd.read_csv(os.path.join(PATH, 'df5.csv'), header=None)[[0,1,2,3,4,5]]

# Run pipeline and see statistics
pipeline = MyClass(df1, df2, matchColumn=0, on=[1,2,3,4,5], method="column", threshold = 0.6) #  --> this means at least 3/5 of the fields must match 
pipeline.setGroundTruth()
pipeline.soundexDfs()

evaluator = DatasetEvaluator(pipeline.df1, pipeline.df2, expected, threshold=3)
evaluator.evaluate()
evaluator.printResults()

NameError: name 'expected' is not defined

In [6]:
80256.47/60/60

22.293463888888887